In [1]:
"""Training script smpl deformation space experiment.
"""
import argparse
import json
import os
import glob
import numpy as np
from collections import defaultdict
import warnings
import time
import trimesh
from utils import render
import torch
import tqdm
from deepdeform.layers.deformation_layer import NeuralFlowDeformer

# Load meshes

In [3]:
# %%capture
# # load meshes
# mesh_files = sorted(glob.glob(os.path.join(data_dir, "*.obj")))
# meshes = []
# for idx, m in enumerate(mesh_files):
#     mesh = trimesh.load(m, process=False)
#     meshes.append(mesh)
# faces = meshes[0].faces
# batch_verts = np.stack([m.vertices for m in meshes], axis=0)  # [n_shapes, n_verts, 3 dims]

data_dir = "data/smpl_animation"
device = torch.device("cuda:2")

d = np.load(os.path.join(data_dir, "meshes.npz"))
faces = d['faces']
batch_verts = d['batch_verts']

n_shapes = len(batch_verts)
verts_t = torch.from_numpy(batch_verts).float().to(device) * 5

# \[Application 1\] Deform between pairs

## Options

In [ ]:
# options
deformer_nf = 16  # number of base feature layers in deformer
atol = 1e-4
rtol = 1e-4
nonlin = 'swish'
adjoint = False
solver = 'rk4'
niter = 1000  # number of iterations
optimizer = torch.optim.Adam
learning_rate = 2e-3
criterion = torch.nn.L1Loss() # torch.nn.MSELoss()
log_per_n_iter = 10
divfree = True  # use divergence free parameterization

## Setup

In [ ]:
# setup model
deformer = NeuralFlowDeformer(latent_size=1, f_width=deformer_nf, s_nlayers=2, 
                              s_width=5, method=solver, nonlinearity=nonlin, arch='imnet',
                              adjoint=adjoint, rtol=rtol, atol=atol, via_hub=False,
                              no_sign_net=False, return_waypoints=False, use_latent_waypoints=False,
                              divfree=divfree)

# for the linear case, we do not require the lat_params to be trainable.
lat_params = torch.linspace(-1, 1, 2)[:, None]  # lat_params = torch.linspace(-1, 1, n_shapes)[:, None]
lat_params = lat_params.unsqueeze(0).to(device)
deformer.to(device)
optim = optimizer(deformer.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 'min', patience=50, factor=0.3, verbose=True, threshold=3e-4)


In [ ]:
# training loop:
v_src = verts_t[0:1]  # [1, nverts, 3]
v_tar = verts_t[4:5]  # [1, nverts, 3]
v_src_tar = torch.cat([v_src, v_tar], dim=0)
v_tar_src = torch.cat([v_tar, v_src], dim=0)
l_src_tar = torch.cat([lat_params, lat_params[:, [1, 0]]], dim=0)

for it in range(niter):
    optim.zero_grad()
    v_prd = deformer(v_src_tar, l_src_tar)  # [nsteps, nverts, 3]
    loss = criterion(v_prd, v_tar_src)
    loss.backward()
    optim.step()
    scheduler.step(loss.item())
    
    if it % log_per_n_iter == 0:
        print(f"Iter {it:4d}: loss {loss.item()}")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

r = trimesh.transformations.rotation_matrix(np.pi/180.*180., [0., 1., 0.])
t = trimesh.transformations.translation_matrix(np.array([-.7, -2.5, 0.]))
s = trimesh.transformations.scale_matrix(7)
m_p = trimesh.Trimesh(v_prd[0].detach().cpu().numpy(), faces).apply_transform(r.dot(t.dot(s)))
m_t = trimesh.Trimesh(v_tar[0].detach().cpu().numpy(), faces).apply_transform(r.dot(t.dot(s)))
m_o = trimesh.Trimesh(v_src[0].detach().cpu().numpy(), faces).apply_transform(r.dot(t.dot(s)))

m_o.visual.vertex_colors = np.ones_like(m_o.vertices)
m_p.visual.vertex_colors = np.ones_like(m_p.vertices)
m_t.visual.vertex_colors = np.ones_like(m_t.vertices)

print(f"Volume of original mesh: {m_o.volume}")
print(f"Volume of deformed mesh: {m_t.volume}")
print(f"Volume of target mesh: {m_p.volume}")

img_org, _ = render.render_trimesh(m_o, color=[1., 1., 1.], res=(512, 512), dist=1.2, intensity=20)
img_prd, _ = render.render_trimesh(m_p, color=[1., .4, .4], res=(512, 512), dist=1.2, intensity=20)
img_tar, _ = render.render_trimesh(m_t, color=[.4, .4, 1.], res=(512, 512), dist=1.2, intensity=20)

fig, axes = plt.subplots(figsize=(24, 8), ncols=3)
axes[0].imshow(img_org)
axes[0].set_title(f"Original Mesh. Volume: {m_o.volume:.4f}")
axes[0].axis('off')
axes[1].imshow(img_prd)
axes[1].set_title(f"Deformed Mesh. Volume: {m_p.volume:.4f}")
axes[1].axis('off')
axes[2].imshow(img_tar)
axes[2].set_title(f"Target   Mesh. Volume: {m_t.volume:.4f}")
axes[2].axis('off')
plt.show()

# \[Application 2\] Sequence optimization

## Options

In [4]:
# options
data_dir = "data/smpl_animation"
deformer_nf = 16  # number of base feature layers in deformer
atol = 1e-4
rtol = 1e-4
nonlin = 'elu'
adjoint = False
solver = 'rk4'
device = torch.device("cuda:2")
niter = 1000  # number of iterations
optimizer = torch.optim.Adam
learning_rate = 1e-3
criterion = torch.nn.L1Loss() # torch.nn.MSELoss()
log_per_n_iter = 10
divfree = True  # use divergence free parameterization

## Setup

In [5]:
# setup model
deformer = NeuralFlowDeformer(latent_size=1, f_width=deformer_nf, s_nlayers=2, 
                              s_width=5, method=solver, nonlinearity=nonlin, arch='imnet',
                              adjoint=adjoint, rtol=rtol, atol=atol, via_hub=False,
                              no_sign_net=False, return_waypoints=True, use_latent_waypoints=True,
                              divfree=divfree)

# for the linear case, we do not require the lat_params to be trainable.
lat_params = torch.linspace(-1, 1, n_shapes)[:, None]
lat_params = lat_params.unsqueeze(0).to(device)
deformer.to(device)
optim = optimizer(deformer.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, 'min', patience=50, factor=0.3, verbose=True, threshold=3e-4)

In [6]:
# training loop:
v_src = verts_t[:1]  # [1, nverts, 3]
v_tar = verts_t[-1:]  # [1, nverts, 3]
v_src_tar_seq = verts_t  # [nsteps, nverts, 3]
v_tar_src_seq = verts_t[list(np.arange(verts_t.shape[0])[::-1])]  # [nsteps, nverts, 3]
v_gt = torch.stack([v_src_tar_seq, v_tar_src_seq], dim=1)  # [nsteps, batch, nverts, 3]
v_src_tar = torch.cat([v_src, v_tar], dim=0)
v_tar_src = torch.cat([v_tar, v_src], dim=0)
l_src_tar = torch.cat([lat_params, lat_params[:, list(np.arange(lat_params.shape[1])[::-1])]], dim=0)
# [batch, nsteps, 1]

for it in range(niter):
    optim.zero_grad()
    v_prd = deformer(v_src_tar, l_src_tar)  # [nsteps, batch, nverts, 3]
    loss = criterion(v_prd, v_gt)
    loss.backward()
    optim.step()
    
    if it % log_per_n_iter == 0:
        print(f"Iter {it:4d}: loss {loss.item()}")
    

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 2; 11.91 GiB total capacity; 10.86 GiB already allocated; 2.62 MiB free; 11.34 GiB reserved in total by PyTorch)

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# for v_p, v_t in zip(v_pred.detach().cpu().numpy(), verts_t[-1].detach().cpu().numpy()):
#     m_p = trimesh.Trimesh(v_p, faces)
#     m_t = trimesh.Trimesh(v_t, faces)
    
#     m_p.visual.vertex_colors = np.ones_like(m_p.vertices)
#     m_t.visual.vertex_colors = np.ones_like(m_t.vertices)
    
#     img_prd, _ = render.render_trimesh(m_p, color=[1., .4, .4], res=(224, 224), dist=1.1, intensity=3)
#     img_tar, _ = render.render_trimesh(m_t, color=[.4, .4, 1.], res=(224, 224), dist=1.1, intensity=3)

#     fig, axes = plt.subplots(figsize=(8, 4), ncols=2)
#     axes[0].imshow(img_prd)
#     axes[1].imshow(img_tar)
#     plt.show()